# Data Preparation for HostIQ App

This notebook prepares two clean CSV files for the Streamlit app:
1. **property_data.csv** - One row per property (with host info)
2. **host_data.csv** - One row per host (aggregated from all their properties)

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports loaded")

## 1. Load Source Data

In [ ]:
# Load the two source files
sample_data = pd.read_csv('sample_data.csv')
review_data = pd.read_csv('property_review_intelligence_report.csv')

print(f"📊 sample_data.csv: {len(sample_data)} rows, {len(sample_data.columns)} columns")
print(f"📊 property_review_intelligence_report.csv: {len(review_data)} rows, {len(review_data.columns)} columns")

## 2. Normalize Sentiment Scores

In [ ]:
def normalize_sentiment(x):
    """Normalize sentiment from [-1, 1] to [0, 1]"""
    if pd.isna(x):
        return x
    if x < 0:
        return (x + 1) / 2
    return x

review_data['Avg_Sentiment_Score'] = review_data['Avg_Sentiment_Score'].apply(normalize_sentiment)
print(f"✅ Sentiment scores normalized to [0, 1] range")
print(f"   Min: {review_data['Avg_Sentiment_Score'].min():.3f}")
print(f"   Max: {review_data['Avg_Sentiment_Score'].max():.3f}")
print(f"   Mean: {review_data['Avg_Sentiment_Score'].mean():.3f}")

## 3. Merge Review Data into Property Data

In [ ]:
# Prepare for merge
review_data['Property_ID'] = review_data['Property_ID'].astype(str)
sample_data['property_id_str'] = sample_data['property_id'].astype(str)

# Mapping from review columns to property columns
review_mapping = {
    'Total_Reviews': 'review_total',
    'Positive_Reviews': 'review_positive', 
    'Negative_Reviews': 'review_negative',
    'Avg_Sentiment_Score': 'review_sentiment',
    'Cleanliness_Rating': 'rating_cleanliness',
    'Communication_Rating': 'rating_communication',
    'Checkin_Rating': 'rating_checkin',
    'Location_Rating': 'rating_location',
    'Value_Rating': 'rating_value',
    'Advantages': 'review_advantages',
    'Disadvantages': 'review_disadvantages',
    'Suggestions': 'review_suggestions'
}

# Update review columns from review_data (source of truth)
for src_col, dst_col in review_mapping.items():
    if src_col in review_data.columns:
        mapping = review_data.set_index('Property_ID')[src_col].to_dict()
        sample_data[dst_col] = sample_data['property_id_str'].map(mapping).combine_first(
            sample_data[dst_col] if dst_col in sample_data.columns else pd.Series()
        )

sample_data = sample_data.drop(columns=['property_id_str'])
print(f"✅ Review data merged into property data")

## 4. Create Property Data Table

In [ ]:
# Calculate price delta
sample_data['price_delta'] = sample_data['prediction'] - sample_data['price']

# Create listing name
sample_data['listing_name'] = (
    sample_data['property_type'].str.replace('_', ' ').str.title() + 
    ' in ' + sample_data['city']
)

# Select and rename columns for property_data.csv
property_data = sample_data[[
    # IDs
    'property_id', 'seller_id',
    # Location
    'city', 'state', 'country', 'lat', 'long',
    # Property details
    'listing_name', 'property_type', 'guests', 'bedrooms', 'beds', 'bathrooms',
    # Amenities
    'has_ac', 'has_wifi', 'has_pool', 'has_kitchen', 'has_parking', 'amenities_count',
    # Pricing
    'price', 'prediction', 'price_delta',
    # Host info
    'is_supperhost', 'host_rating', 'host_response_rate', 'host_number_of_reviews', 'hosts_year',
    # Reviews
    'review_total', 'review_positive', 'review_negative', 'review_sentiment',
    # Ratings
    'rating_cleanliness', 'rating_communication', 'rating_checkin', 'rating_location', 'rating_value',
    # Review insights
    'review_advantages', 'review_disadvantages', 'review_suggestions',
    # Location scores
    'walk_score', 'poi_total', 'tourism_density'
]].copy()

# Rename columns for clarity
property_data = property_data.rename(columns={
    'seller_id': 'host_id',
    'is_supperhost': 'is_superhost'
})

print(f"✅ property_data created: {len(property_data)} rows, {len(property_data.columns)} columns")
print(f"\nColumns: {list(property_data.columns)}")

## 5. Create Host Data Table (Aggregated)

In [ ]:
# Aggregate by host
host_data = property_data.groupby('host_id').agg({
    # Count
    'property_id': 'count',
    # Host info (same for all properties of host)
    'is_superhost': 'first',
    'host_rating': 'first',
    'host_response_rate': 'first',
    'hosts_year': 'first',
    'host_number_of_reviews': 'first',
    # Reviews - sum
    'review_total': 'sum',
    'review_positive': 'sum',
    'review_negative': 'sum',
    # Sentiment - mean
    'review_sentiment': 'mean',
    # Ratings - mean
    'rating_cleanliness': 'mean',
    'rating_communication': 'mean',
    'rating_checkin': 'mean',
    'rating_location': 'mean',
    'rating_value': 'mean',
    # Pricing - mean
    'price': 'mean',
    'prediction': 'mean',
    'price_delta': 'mean',
    # Location - mean
    'lat': 'mean',
    'long': 'mean',
    'walk_score': 'mean',
    # Amenities - mean
    'amenities_count': 'mean',
    # Property type - mode
    'property_type': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
    # Review insights - combine unique values
    'review_advantages': lambda x: '; '.join([str(a) for a in x.dropna().unique()[:5]]),
    'review_disadvantages': lambda x: '; '.join([str(d) for d in x.dropna().unique()[:5]]),
    'review_suggestions': lambda x: '; '.join([str(s) for s in x.dropna().unique()[:3]]),
    # Property IDs list
    'property_id': lambda x: ','.join([str(p) for p in x])
}).reset_index()

# Fix the property_id column (it was overwritten by the last aggregation)
# Re-aggregate to get both count and list
host_data = property_data.groupby('host_id').agg({
    'property_id': ['count', lambda x: ','.join([str(p) for p in x])],
    'is_superhost': 'first',
    'host_rating': 'first',
    'host_response_rate': 'first',
    'hosts_year': 'first',
    'host_number_of_reviews': 'first',
    'review_total': 'sum',
    'review_positive': 'sum',
    'review_negative': 'sum',
    'review_sentiment': 'mean',
    'rating_cleanliness': 'mean',
    'rating_communication': 'mean',
    'rating_checkin': 'mean',
    'rating_location': 'mean',
    'rating_value': 'mean',
    'price': 'mean',
    'prediction': 'mean',
    'price_delta': 'mean',
    'lat': 'mean',
    'long': 'mean',
    'walk_score': 'mean',
    'amenities_count': 'mean',
    'property_type': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
    'review_advantages': lambda x: '; '.join([str(a) for a in x.dropna().unique()[:5]]),
    'review_disadvantages': lambda x: '; '.join([str(d) for d in x.dropna().unique()[:5]]),
    'review_suggestions': lambda x: '; '.join([str(s) for s in x.dropna().unique()[:3]])
}).reset_index()

# Flatten column names
host_data.columns = [
    'host_id',
    'property_count', 'property_ids',
    'is_superhost',
    'host_rating',
    'host_response_rate',
    'hosts_year',
    'host_number_of_reviews',
    'total_reviews',
    'positive_reviews',
    'negative_reviews',
    'avg_sentiment',
    'avg_rating_cleanliness',
    'avg_rating_communication',
    'avg_rating_checkin',
    'avg_rating_location',
    'avg_rating_value',
    'avg_price',
    'avg_prediction',
    'avg_price_delta',
    'avg_lat',
    'avg_long',
    'avg_walk_score',
    'avg_amenities_count',
    'main_property_type',
    'combined_advantages',
    'combined_disadvantages',
    'combined_suggestions'
]

# Calculate quality score
host_data['quality_score'] = host_data['avg_sentiment'].fillna(0.5)

print(f"✅ host_data created: {len(host_data)} rows, {len(host_data.columns)} columns")
print(f"\nColumns: {list(host_data.columns)}")

## 6. Export to CSV

In [ ]:
# Export property data
property_data.to_csv('property_data.csv', index=False)
print(f"✅ Exported: property_data.csv ({len(property_data)} rows)")

# Export host data
host_data.to_csv('host_data.csv', index=False)
print(f"✅ Exported: host_data.csv ({len(host_data)} rows)")

## 7. Verify Data

In [ ]:
print("=" * 60)
print("DATA VERIFICATION")
print("=" * 60)

print(f"\n📊 property_data.csv")
print(f"   Rows: {len(property_data)}")
print(f"   Unique properties: {property_data['property_id'].nunique()}")
print(f"   Unique hosts: {property_data['host_id'].nunique()}")

print(f"\n📊 host_data.csv")
print(f"   Rows: {len(host_data)}")
print(f"   Total properties (sum): {host_data['property_count'].sum()}")
print(f"   Superhosts: {host_data['is_superhost'].sum()}")

print(f"\n✅ Verification passed!")
print(f"   - All {len(property_data)} properties mapped to {len(host_data)} hosts")

In [ ]:
# Preview the data
print("\n📋 property_data.csv preview:")
display(property_data.head(3))

print("\n📋 host_data.csv preview:")
display(host_data.head(3))